In [30]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
from num2words import num2words

import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math
import nltk
nltk.download('stopwords')
nltk.download('punkt')
# %load_ext autotime

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ppriya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ppriya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [50]:
def print_doc(id, dataset):
    #print(dataset[id])
    file = open(dataset[id][0], 'r', encoding='cp1250')
    text = file.read().strip()
    file.close()
    #print(text)

In [32]:
def convert_lower_case(data):
    return np.char.lower(data)

In [33]:
def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text

In [34]:
def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

In [35]:
def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

In [36]:
def stemming(data):
    stemmer= PorterStemmer()
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return new_text

In [37]:
def convert_numbers(data):
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        try:
            w = num2words(int(w))
        except:
            a = 0
        new_text = new_text + " " + w
    new_text = np.char.replace(new_text, "-", " ")
    return new_text

In [38]:
def preprocess(data):
    data = convert_lower_case(data)
    data = remove_punctuation(data) #remove comma seperately
    data = remove_apostrophe(data)
    data = remove_stop_words(data)
    data = convert_numbers(data)
    data = stemming(data)
    data = remove_punctuation(data)
    data = convert_numbers(data)
    data = stemming(data) #needed again as we need to stem the words
    data = remove_punctuation(data) #needed again as num2word is giving few hypens and commas fourty-one
    data = remove_stop_words(data) #needed again as num2word is giving stop words 101 - one hundred and one
    return data

In [51]:
def genDataSet(start, end):
    dataset = []

    c = False

    folders = [x[0] for x in os.walk('C:\\Users\\ppriya\\OneDrive - NetApp Inc\\Downloads\\stories\\stories')]

    for i in folders:
        file = open(i+"\\index.html", 'r')
        text = file.read().strip()
        file.close()

        file_name = re.findall('><A HREF="(.*)">', text)
        file_title = re.findall('<BR><TD> (.*)\n', text)

        if c == False:
            file_name = file_name[2:]
            c = True

        #print(len(file_name), len(file_title))

        for j in range(len(file_name)):
            dataset.append((str(i) +"/"+ str(file_name[j]), file_title[j]))
    return dataset[start:end]
    
def extractData(dataset) :
    processed_text = []
    processed_title = []

    for i in dataset:
        file = open(i[0], 'r', encoding="utf8", errors='ignore')
        text = file.read().strip()
        file.close()

        processed_text.append(word_tokenize(str(preprocess(text))))
        processed_title.append(word_tokenize(str(preprocess(i[1]))))
    return processed_text, processed_title

In [63]:
def genDocFreqForAllDocs(processed_text, processed_title, start, end) :
    DF = {}

    for i in range(len(processed_text)):
        tokens = processed_text[i ]
        for w in tokens:
            try:
                DF[w].add(i + start)
            except:
                DF[w] = {i + start}

        tokens = processed_title[i]
        for w in tokens:
            try:
                DF[w].add(i + start)
            except:
                DF[w] = {i + start}
    for i in DF:
        DF[i] = len(DF[i])
    return DF

In [17]:
def doc_freq(DF, word):
    c = 0
    try:
        c = DF[word]
    except:
        pass
    return c

In [96]:
def getDocumentMetadata(processed_text, processed_title, start, end) :
    DMText = {}
    DMTitle = {}

    for i in range(len(processed_text)):
        tokens = processed_text[i]
        counter = Counter(tokens)
        for token in np.unique(tokens):
            try:
                DMText[i + start].add(token, counter[token])
            except:
                DMText[i + start] = {token, counter[token]}


    for i in range(len(processed_text)):
        tokens = processed_title[i]
        counter = Counter(tokens)
        for token in np.unique(tokens):
            try:
                DMTitle[i+ start].add(token, counter[token])
            except:
                DMTitle[i + start] = {token, counter[token]}

    return DMText, DMTitle

In [19]:
def termFreq(DM, term, doc_id) :
    tf = 0
    try:
        tf = DM[doc_id][term]
    except:
        pass
    return tf

In [24]:
def tfidf(PL, DMTitle, DMText, term, doc_id):
    alpha = 0.3
    tf = (1-alpha) * termFreq(DMTitle, term, doc_id) + alpha * termFreq(DMText, term, doc_id)
    df = docFreq(PL, token)
    idf = np.log((N+1)/(df+1))
    return td*idf

In [94]:
def buildIndex(start, end) :
    dataset = genDataSet(start, end)
    processed_text, processed_title = extractData(dataset)
    DF = genDocFreqForAllDocs(processed_text, processed_title, start, end)
    DMText, DMTitle = getDocumentMetadata(processed_text, processed_title, start, end)
    return DF, DMText, DMTitle
    #tfidf = calcDFIDF(DF, processed_text, processed_title)
    
def plMerge(pl1, pl2):
    #Merge dictionaries and keep values of common keys in list
    pl = {}
    for key, value in pl1.items():
        if key in pl2:
            pl[key] = pl1[key] + pl2[key]
        else :
            pl[key] = pl1[key]
    for key, value in pl2.items():
        if key not in pl1:
            pl[key] = pl2[key]
    return pl

def mergeIndices(pl1, pl2, dmtext1, dmtext2, dmtitle1, dmtitle2): #pl - posting list, dm - document metadata
    dmtext = {}
    dmtitle = {}
    pl = {}
    dmtext = {**dmtext1, **dmtext2}
    dmtitle = {**dmtitle1, **dmtitle2}
    pl = plMerge(pl1, pl2)
    return pl, dmtext, dmtitle

def queryfunc(k, query):
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))

    print("My Query Function")
    print("\nQuery:", query)
    print("")
    print(tokens)

    PL, DMText, DMTitle = buildIndex(title)
    query_weights = {}
    for w in tokens:
        for d in DF[w]:
            try:
                query_weights[d] += tfidf(PL, DMText, DMTitle, w, d)
            except:
                query_weights[d] = tdidf(PL, DMText, DMTitle, w, d)
    query_weights = sorted(query_weights.items(), key=lambda x: x[1], reverse=True)

    print("")
    
    l = []
    
    for i in query_weights[:k]:
        l.append(i[0])
    
    #print(l)
    return l
import time

def testIndexes():
    
    start = time.process_time()
    DF1, DMText1, DMTitle1 = buildIndex(0, 100)
    print(time.process_time() - start)
    start = time.process_time()
    DF2, DMText2, DMTitle2 = buildIndex(100, 200)
    print(time.process_time() - start)
    DF, DMText, DMTitle = buildIndex(0, 200)
    start = time.process_time()
    mergedDF, mergedDMText, mergedDMTitle = mergeIndices(DF1, DF2, DMText1, DMText2, DMTitle1, DMTitle2)
    print(time.process_time() - start)
    print(mergedDMText == DMText)
    print (mergedDMTitle == DMTitle)
    print(DF == mergedDF)
    print(len(DMText1.keys))

In [97]:
import timeit
testIndexes()

26.234375
27.578125
0.0
True
True
True
